In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
import scipy.stats as stats
import glob

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) #this was relevant for the String-boolean Int-boolean comparison in the Residue vs Atom double filtering below (god I miss SQL...)


In [2]:
#There is one thing to note before trying this with your own data.
#
#I had to modify a bit the .dat files in order to be able to be read by pandas properly.
#
#Here bellow i'm going to paste some commands I used, normally it is just deleting one "#" which is at the begining of the file in the column names.
#
# This is the unix command I used for modifying the files
#sed -i -e 's,# Residue/ Residue/g' */raw-data/P_RMSF.dat
#sed -i -e 's,file_name=.*/,file_name=,' file
#sed -i s/# Frame/ Frame/g' */raw-data/L-Properties.dat
#sed -i s/# frame/ frame/g' */raw-data/PL_RMSD.dat


# IF YOU WORK ON MAC SED IS NOT YOUR FRIEND. I used a lazy Perl solution on the one-liner
# Please try a variation of
# perl -pi -w -e 's/  # Frame#/Frame/g;' */raw-data/PL-Contacts_*.dat
# perl -pi -w -e 's/ Residue#/Residue/g;' */raw-data/PL-Contacts_*.dat

In [29]:
# My poor less fancy importing scheme for multiple files =P
wrkdir = 'C:\\Users\\glauc\\OneDrive\\FBPase\\Results-gmf\\SID_v2\\Results' #You have to define where the folders and subfolder will be
os.chdir(wrkdir)
name="202_TB4" #change here the name of the system, so it can pick all the files automatically

#Those features are stratified with the number of residue only
hydrophobic=pd.read_table('C:\\Users\\glauc\\OneDrive\\FBPase\\Results-gmf\\SID_v2\\202\\Sub-Chain B\\raw-data\\PL-Contacts_Hydrophobic.dat'.format(wrkdir, name), sep='\s+')
ionic=pd.read_table('C:\\Users\\glauc\\OneDrive\\FBPase\\Results-gmf\\SID_v2\\202\\Sub-Chain B\\raw-data\\PL-Contacts_Ionic.dat'.format(wrkdir, name), sep='\s+')

#Those features need the number of residue and atom for proper separation
pipi=pd.read_table('C:\\Users\\glauc\\OneDrive\\FBPase\\Results-gmf\\SID_v2\\202\\Sub-Chain B\\raw-data\\PL-Contacts_Pi-Pi.dat'.format(wrkdir, name), sep='\s+')
hbond=pd.read_table('C:\\Users\\glauc\\OneDrive\\FBPase\\Results-gmf\\SID_v2\\202\\Sub-Chain B\\raw-data\\PL-Contacts_HBond.dat'.format(wrkdir, name), sep='\s+')
water=pd.read_table('C:\\Users\\glauc\\OneDrive\\FBPase\\Results-gmf\\SID_v2\\202\\Sub-Chain B\\raw-data\\PL-Contacts_WaterBridge.dat'.format(wrkdir, name), sep='\s+')


In [30]:
#HYDROGEN BOND INTERACTIONS
hbond_unique=hbond.drop_duplicates(['Residue','AtomName'])
hbond_unique = hbond_unique.reset_index(drop=True)
hbond_db = [{'Type':0,'ResName':0,'Residue':0, 'AtomName':0,'Frequency':0}] #I setup an interaction Type variable for later help with the plotting
x=0
while x<hbond_unique["Residue"].size:
    
    freq = np.sum((hbond["Residue"]==hbond_unique["Residue"][x])|(hbond["AtomName"]==hbond_unique["AtomName"][x]))/(hbond["Residue"].size-1)*100
    if(freq>10): #Filtering to export only interactions with at least 10% frequency
        tmp_db= {'Type':"hbond",'ResName':hbond_unique["ResName"][x],'Residue':hbond_unique["Residue"][x],'AtomName':hbond_unique["AtomName"][x], 'Frequency':freq} #generates an entry in the dictionary with the residue number and Freq%
        hbond_db.append(tmp_db) #Appends the entry in the instanced dictionary
    x=x+1
    
print("hydrogen bond by Residue and AtomName number")
hbond_df=pd.DataFrame(hbond_db)
hbond_df=hbond_df.loc[~(hbond_df==0).all(axis=1)]
print(pd.DataFrame(hbond_df))

#HYDROPHOBIC INTERACTIONS DEFINITIONS
hydrophobic_unique=hydrophobic.drop_duplicates(['Residue'])
hydrophobic_unique = hydrophobic_unique.reset_index(drop=True)
hydrophobic_db = [{'Type':0,'ResName':0,'Residue':0 , 'Frequency': 0}] #I setup an interaction Type variable for later help with the plotting

x=0 #resetting the counter!!
while x<hydrophobic_unique["Residue"].size:
    freq = np.sum((hydrophobic["Residue"]==hydrophobic_unique["Residue"][x]))/(hydrophobic["Residue"].size-1)*100
    if(freq>10): #Filtering to export only interactions with at least 10% frequency
        tmp_db= {'Type':"hydrophobic",'ResName':hydrophobic_unique["ResName"][x],'Residue':hydrophobic_unique["Residue"][x],'Frequency':freq} #generates an entry in the dictionary with the residue number and Freq%
        hydrophobic_db.append(tmp_db) #Appends the entry in the instanced dictionary
    x=x+1
print("Hydrophobic interactions by Residue number")
hydrophobic_df=pd.DataFrame(hydrophobic_db)
hydrophobic_df=hydrophobic_df.loc[~(hydrophobic_df==0).all(axis=1)]
print(pd.DataFrame(hydrophobic_df))

#IONIC INTERACTIONS
ionic_unique=ionic.drop_duplicates(['Residue'])
ionic_unique = ionic_unique.reset_index(drop=True)
ionic_db = [{'Type':0,'ResName':0,'Residue':0 , 'Frequency': 0}] #I setup an interaction Type variable for later help with the plotting

x=0 #resetting the counter!!
while x<ionic_unique["Residue"].size:
    freq = np.sum((ionic["Residue"]==ionic_unique["Residue"][x]))/(ionic["Residue"].size-1)*100
    if(freq>10): #Filtering to export only interactions with at least 10% frequency
        tmp_db= {'Type':"ionic",'ResName':ionic_unique["ResName"][x],'Residue':ionic_unique["Residue"][x],'Frequency':freq} #generates an entry in the dictionary with the residue number and Freq%
        ionic_db.append(tmp_db) #Appends the entry in the instanced dictionary
    x=x+1
print("Ionic interactions by Residue number")
ionic_df=pd.DataFrame(ionic_db)
ionic_df=ionic_df.loc[~(ionic_df==0).all(axis=1)]
print(pd.DataFrame(ionic_df))

#PI-PI INTERACTIONS
pipi_unique=pipi.drop_duplicates(['Residue'])
pipi_unique = pipi_unique.reset_index(drop=True)
pipi_db = [{'Type':0,'ResName':0,'Residue':0 , 'Frequency': 0}] #I setup an interaction Type variable for later help with the plotting

x=0 #resetting the counter!!
while x<pipi_unique["Residue"].size:
    freq = np.sum((pipi["Residue"]==pipi_unique["Residue"][x]))/(pipi["Residue"].size-1)*100
    if(freq>10): #Filtering to export only interactions with at least 10% frequency
        tmp_db= {'Type':"pi-pi",'ResName':pipi_unique["ResName"][x],'Residue':pipi_unique["Residue"][x],'Frequency':freq} #generates an entry in the dictionary with the residue number and Freq%
        pipi_db.append(tmp_db) #Appends the entry in the instanced dictionary
    x=x+1
print("pi-pi interactions by Residue number")
pipi_df=pd.DataFrame(pipi_db)
pipi_df=pipi_df.loc[~(pipi_df==0).all(axis=1)]
print(pd.DataFrame(pipi_df))

#WATER INTERACTIONS
water_unique=water.drop_duplicates(['Residue'])
water_unique = water_unique.reset_index(drop=True)
water_db = [{'Type':0,'ResName':0,'Residue':0 , 'Frequency': 0}] #I setup an interaction Type variable for later help with the plotting

x=0 #resetting the counter!!
while x<water_unique["Residue"].size:
    freq = np.sum((water["Residue"]==water_unique["Residue"][x]))/(water["Residue"].size-1)*100
    if(freq>10): #Filtering to export only interactions with at least 10% frequency
        tmp_db= {'Type':"water",'ResName':water_unique["ResName"][x],'Residue':water_unique["Residue"][x],'Frequency':freq} #generates an entry in the dictionary with the residue number and Freq%
        water_db.append(tmp_db) #Appends the entry in the instanced dictionary
    x=x+1
print("water-mediated interactions by Residue number")
water_df=pd.DataFrame(water_db)
water_df=water_df.loc[~(water_df==0).all(axis=1)]
print(pd.DataFrame(water_df))

hydrogen bond by Residue and AtomName number
     Type ResName  Residue AtomName  Frequency
1   hbond     ASP      181      OD1  18.572616
2   hbond     ASP      183      OD1  20.092626
3   hbond     TYR      114       HH  11.411946
4   hbond     ARG      159       HE  25.495784
5   hbond     ARG      159     HH21  25.495784
6   hbond     ARG      161     HH12  22.895143
7   hbond     ARG      161     HH22  23.132645
8   hbond     ASP      181      OD2  20.211376
9   hbond     ASP      183      OD2  17.432609
10  hbond     TYR      114       OH  11.411946
11  hbond     ARG      159     HH22  36.872105
12  hbond     ASP      183        H  20.627004
13  hbond     ASP       82      OD1  13.169457
14  hbond     ARG      230     HH12  12.635079
15  hbond     ARG      230     HH22  12.112576
16  hbond     ARG      161     HH11  22.764517
17  hbond     ARG      161       HE  33.950837
18  hbond     ARG      161     HH21  36.468353
Hydrophobic interactions by Residue number
          Type ResN

In [31]:
#In case you want to have different styles : https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
frames = pd.concat([hbond_df,water_df,hydrophobic_df,ionic_df,pipi_df])
#frames.to_csv('interactions.tsv', sep = '\t')
print(frames)
frames.to_csv('interactions_{}.tsv'.format(name), sep='\t')




           Type ResName  Residue AtomName  Frequency
1         hbond     ASP      181      OD1  18.572616
2         hbond     ASP      183      OD1  20.092626
3         hbond     TYR      114       HH  11.411946
4         hbond     ARG      159       HE  25.495784
5         hbond     ARG      159     HH21  25.495784
6         hbond     ARG      161     HH12  22.895143
7         hbond     ARG      161     HH22  23.132645
8         hbond     ASP      181      OD2  20.211376
9         hbond     ASP      183      OD2  17.432609
10        hbond     TYR      114       OH  11.411946
11        hbond     ARG      159     HH22  36.872105
12        hbond     ASP      183        H  20.627004
13        hbond     ASP       82      OD1  13.169457
14        hbond     ARG      230     HH12  12.635079
15        hbond     ARG      230     HH22  12.112576
16        hbond     ARG      161     HH11  22.764517
17        hbond     ARG      161       HE  33.950837
18        hbond     ARG      161     HH21  36.